In [38]:
import pandas as pd
import numpy as np
import string
import os
import string
from fuzzywuzzy import fuzz, process
import sklearn
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
central_data = pd.read_csv('/Users/kchua/Documents/Data_Stories/2023-10-tx-produced-water-spills/data/central_cleaned.csv')
central_data.head()

,dist,district_edit,date_called_in,date_called_in_edit,date_of_spill,date_of_spill_edit,spill_number,rrc_job_number,operator_rp,operator_edit,...,cleanup_oversight_austin,status,comments,compliance_date,file,sheet,cleanup_criteria,cleanup_oversight,rrc_job_number_2,my_of_spill
0,04,4,2013-01-13,2013-01-13,2013-01-13,2013-01-13,04-1238,2013-381,Headington Oil Company,HEADINGTON OIL,...,NaN,assessment phase,Leak in gathering line DS,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
1,06,6,2013-01-06,2013-01-06,2013-01-05,2013-01-05,1291,2013-06-195,"Quantum Resources Management, LLC",QUANTUM RESOURCES,...,41284,JH,NaN,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
2,7C,7C,2013-01-22,2013-01-22,2013-01-22,2013-01-22,1829,324,EOG Resources,EOG Resources,...,No,assessment phase,Dump valve misfired,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
3,7C,7C,2013-01-14,2013-01-14,2013-01-13,2013-01-13,1830,191,Pioneer Natural Res. USA Inc.,Pioneer Natural Resources (acq. by ExxonMobil ...,...,No,cleanup phase,"water leg plugged up water went in oil tank, r...",NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01
4,7C,7C,2013-01-14,2013-01-14,2013-01-05,2013-01-05,1832,192,"Molopo Energy Texas, LLC","MOLOPO ENERGY TEXAS, LLC",...,No,cleanup phase,heater treater was not started ran prod tank over,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01


In [3]:
data = central_data

In [4]:
# Cleaning and fuzzy matching the source values

# Remove punctuation and whitespace
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

def remove_whitespace(text):
    text = ' '.join(text.split())
    return text

# Lowercasing
def impute_nulls(col): 
    clean_col = col.str.lower()
    clean_col = clean_col.fillna('null')
    clean_col = clean_col.replace('', 'null')
    clean_col = clean_col.apply(lambda x: x.strip())
    clean_col = clean_col.apply(remove_whitespace)
    return clean_col

# Functions for typo correction
def correct_typos(x, typo_dict): 
    if x in typo_dict: 
        return typo_dict[x]
    else: 
        return x

In [5]:
# Functions for typo exploration

def find_best_match(value, choices, threshold=80):
    match, score = process.extractOne(value, choices)
    return match if score >= threshold else value

def plot_kcluster_elbow(word_list):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(word_list)

    # Experiment with a range of clusters
    max_clusters = 10
    distortions = []

    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)

    # Plot the Elbow curve
    plt.plot(range(1, max_clusters + 1), distortions, marker='o')
    plt.title('Elbow Method for Optimal k')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Distortion')
    plt.show()

def show_kclusters(word_list, optimal_clusters):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(word_list)

    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    kmeans.fit(X)

    word_list_clusters = list(zip(word_list, kmeans.labels_))

    for cluster_id in range(optimal_clusters):
        cluster_words = [word for word, label in word_list_clusters if label == cluster_id]
        print(f"Cluster {cluster_id + 1}: {cluster_words}")
        print()
        print()

In [6]:
# Limit rows for testing
col_name = 'operator_edit'
new_col_name = col_name + '_kc'
data[new_col_name] = impute_nulls(data[col_name])

In [7]:
counts = data[new_col_name].value_counts()
counts

operator_edit_kc
occidental petroleum                                   997
pioneer natural resources (acq. by exxonmobil 2023)    596
xto energy (exxonmobil subsidiary)                     589
endeavor energy                                        393
energen (acq. by diamondback 2019)                     310
                                                      ... 
canyon creek energy texas, llc                           1
arcturus well services llc                               1
stampede energy resources, llc                           1
manhattan petroleum, inc.                                1
pinon operating llc                                      1
Name: count, Length: 1139, dtype: int64

In [8]:
duplicated_values = counts[counts != 1].index.tolist()
#print(duplicated_values)
print(len(duplicated_values))
non_duplicated_values = counts[counts == 1].index.tolist()
print(len(non_duplicated_values))

499
640


In [9]:
#data = data.sample(50)
print(data.shape)
data.head()

(10326, 57)


,dist,district_edit,date_called_in,date_called_in_edit,date_of_spill,date_of_spill_edit,spill_number,rrc_job_number,operator_rp,operator_edit,...,status,comments,compliance_date,file,sheet,cleanup_criteria,cleanup_oversight,rrc_job_number_2,my_of_spill,operator_edit_kc
0,04,4,2013-01-13,2013-01-13,2013-01-13,2013-01-13,04-1238,2013-381,Headington Oil Company,HEADINGTON OIL,...,assessment phase,Leak in gathering line DS,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,headington oil
1,06,6,2013-01-06,2013-01-06,2013-01-05,2013-01-05,1291,2013-06-195,"Quantum Resources Management, LLC",QUANTUM RESOURCES,...,JH,NaN,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,quantum resources
2,7C,7C,2013-01-22,2013-01-22,2013-01-22,2013-01-22,1829,324,EOG Resources,EOG Resources,...,assessment phase,Dump valve misfired,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,eog resources
3,7C,7C,2013-01-14,2013-01-14,2013-01-13,2013-01-13,1830,191,Pioneer Natural Res. USA Inc.,Pioneer Natural Resources (acq. by ExxonMobil ...,...,cleanup phase,"water leg plugged up water went in oil tank, r...",NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,pioneer natural resources (acq. by exxonmobil ...
4,7C,7C,2013-01-14,2013-01-14,2013-01-05,2013-01-05,1832,192,"Molopo Energy Texas, LLC","MOLOPO ENERGY TEXAS, LLC",...,cleanup phase,heater treater was not started ran prod tank over,NaN,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,"molopo energy texas, llc"


In [10]:
typo_corrections = {
}

col_threshold = 80

In [49]:
#sampled_names = data[col_name].sample(n=30, replace=False).unique()
# Print the sampled names
#for name in sampled_names:
#    print(name)

oil_company_stopwords = [
    'energy', 
    'petroleum', 
    'resource',
    'resources',
    'llc',
    'oil', 
    'gas',
    'company',
    'operating',
    'inc',
    'service',
    'corp',
    'products',
    'producing', 
    'production',
    'ltd',
    'exploration'
]

def clean_company_name(company_name):
    if pd.isna(company_name):
        return ""
    # Remove punctuation
    company_name = company_name.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stop words
    words = company_name.split()
    filtered_words = [word.lower() for word in words if word.lower() not in oil_company_stopwords]
    return ' '.join(filtered_words)

# Apply the function to the 'company_names' column
data[new_col_name + '_clean'] = data[new_col_name].apply(clean_company_name)

In [50]:
data[col_name + '_clean']

0                                    headington
1                                       quantum
2                                           eog
3        pioneer natural acq by exxonmobil 2023
4                                  molopo texas
                          ...                  
10321                                    apache
10322                                crossroads
10323                                     walsh
10324                               bridwell co
10325                                 lime rock
Name: operator_edit_clean, Length: 10326, dtype: object

In [53]:
# Apply the custom function to populate a new column
#non_duplicated_data = data.loc[data[new_col_name].isin(non_duplicated_values)]
#%timeit data.loc[data[new_col_name].isin(non_duplicated_values), new_col_name + '_best_match'] = data.loc[data[new_col_name].isin(non_duplicated_values)].apply(lambda row: row_find_best_match(row, new_col_name, duplicated_values, threshold=col_threshold), axis=1)
clean_company_unique_vals = data[data[new_col_name].isin(duplicated_values)][new_col_name + '_clean'].unique()
len(clean_company_unique_vals)

472

In [54]:
%timeit data[new_col_name + '_best_match'] = data[new_col_name].apply(lambda x: find_best_match(x, clean_company_unique_vals, threshold=col_threshold))

7min 45s ± 3.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
data.to_csv('data.csv')

In [ ]:
# Check discrepancies
discrepancies = data[[col_name, new_col_name, new_col_name + '_best_match']]
discrepancies = discrepancies[discrepancies[new_col_name] != discrepancies[new_col_name + '_best_match']]
discrepancies = discrepancies.drop_duplicates()
discrepancies.sort_values(new_col_name + '_best_match').head(20)

In [ ]:
# Maybe use some kind of entity recognition - or just strip out company stop words

In [57]:
data_x = pd.read_csv('data.csv')
data_x.head()

,Unnamed: 0,dist,district_edit,date_called_in,date_called_in_edit,date_of_spill,date_of_spill_edit,spill_number,rrc_job_number,operator_rp,...,file,sheet,cleanup_criteria,cleanup_oversight,rrc_job_number_2,my_of_spill,operator_edit_kc,operator_edit_clean,operator_edit_kc_clean,operator_edit_kc_best_match
0,0,04,4,2013-01-13,2013-01-13,2013-01-13,2013-01-13,04-1238,2013-381,Headington Oil Company,...,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,headington oil,headington,headington,g g
1,1,06,6,2013-01-06,2013-01-06,2013-01-05,2013-01-05,1291,2013-06-195,"Quantum Resources Management, LLC",...,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,quantum resources,quantum,quantum,quantum
2,2,7C,7C,2013-01-22,2013-01-22,2013-01-22,2013-01-22,1829,324,EOG Resources,...,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,eog resources,eog,eog,eog
3,3,7C,7C,2013-01-14,2013-01-14,2013-01-13,2013-01-13,1830,191,Pioneer Natural Res. USA Inc.,...,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,pioneer natural resources (acq. by exxonmobil ...,pioneer natural acq by exxonmobil 2023,pioneer natural acq by exxonmobil 2023,pioneer natural acq by exxonmobil 2023
4,4,7C,7C,2013-01-14,2013-01-14,2013-01-05,2013-01-05,1832,192,"Molopo Energy Texas, LLC",...,logs/master/pattern1/All Dist Spill Log - FY 2...,January 13,NaN,NaN,NaN,2013-01-01,"molopo energy texas, llc",molopo texas,molopo texas,texas american


In [69]:
discrepancies = data_x[[col_name, new_col_name, new_col_name + '_best_match']]
discrepancies = discrepancies[discrepancies[new_col_name] != discrepancies[new_col_name + '_best_match']]
discrepancies = discrepancies.drop_duplicates()
discrepancies = discrepancies.sort_values(
    ,
    new_col_name)
discrepancies.to_csv('discrepancies.csv')
#print(discrepancies.shape)

SyntaxError: invalid syntax (573615421.py, line 5)